# Module 2: Creating and Evaluating Solutions 

`
Rev Date           By       Description
PA1 2020-02-16     akirmak  Modified & extended version of PersonalizePoC created by Chris King (github: chrisking@)
`

In this module, we will create 3 solution using three Amazon Personalize Recipes: 
1. HRNN - Personalization
1. SIMS(Similar Items) - Given one item, what other items are also interacted with by users.
1. Personalized-Ranking - Takes a collection of items and then orders them in probable order of interest using an HRNN-like approach.

We will also evaluate the performance of each solution by analyzing several rank-aware metrics. 


## Creating Solutions

This nobeook will cover creating the following solutions:

1. HRNN
1. SIMS
1. Personalized-Ranking

After that the metrics will be explained and another notebook will showcase how to interact with the Solutions once they are deployed into a Campaign.

The first step is to reload the imports and the stored variables from the previous notebooks.

In [1]:
import boto3
from time import sleep
import subprocess
import pandas as pd
import json
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from datetime import datetime

In [2]:
%store -r

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In Amazon Personalize a trained model is called a Solution, each Solution can have many specific versions that relate to a given volume of data when the model was trained.

To begin we will list all the recipies that are supported, a recipie is an algorithm that has not been trained on your data yet. After listing you'll select one and use that to build your model.


In [21]:
list_recipes_response = personalize.list_recipes()
for recipe in list_recipes_response['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims


That is just a JSON representation of all of the algorithms that we have mentioned already. 

Next we will select a particular algorithm then build a model with it.

### HRNN


HRNN is one of the more advanced recommendation models that you can use and it allows for things like real-time updates of recommendations based on user behavior. It also tends to out perform other approaches like collaborative filtering. We will kick this job off first as it takes the longest to complete.

#### Select Recipe

In [5]:
HRNN_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

#### Create and Wait for Solution
First you will create the solution with the API, then you will create a version. 

Note the solution is just a label kind of identifier, you'll also need to create a version which is the actual trained model.

In [6]:
hrnn_create_solution_response = personalize.create_solution(
    name = "personalize-poc-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = HRNN_recipe_arn
)

hrnn_solution_arn = hrnn_create_solution_response['solutionArn']
print(json.dumps(hrnn_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:924376141954:solution/personalize-poc-hrnn",
  "ResponseMetadata": {
    "RequestId": "e9485e94-c76d-40c3-af75-677642bd0153",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 15 Feb 2020 13:29:33 GMT",
      "x-amzn-requestid": "e9485e94-c76d-40c3-af75-677642bd0153",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the Solution Version

This process will actually take a while to complete, upwards of 25 minutes on. Normally there would be while loops to poll until the task is completed. However the task would block other cells from executing and the goal here is to create many models and deploy them quickly. Below there are instructions to viewing the progress in browser. After creating all of the solution versions go there and watch for updates.

In [9]:
hrnn_create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

In [14]:
hrnn_solution_version_arn = hrnn_create_solution_version_response['solutionVersionArn']
print(json.dumps(hrnn_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:924376141954:solution/personalize-poc-hrnn/00987cb2",
  "ResponseMetadata": {
    "RequestId": "dceef793-0264-43a6-9912-b531ad0999b7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 15 Feb 2020 13:31:04 GMT",
      "x-amzn-requestid": "dceef793-0264-43a6-9912-b531ad0999b7",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS


SIMS is one of the longest running algorithms within Amazon for recommendation systems. A core use case for it is when you have one item and you want to recommend items that have been intereacted with in similar ways over your entire user base(not personalized). Sometimes this leads to recommending mostly popular items, so there is a hyperparameter that can be tweaked that will reduce the popular items in your results. 

Just as last time we start by selecting the recipie:

#### Select Recipe

In [7]:
## SIMS:
SIMS_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

#### Create and Wait for Solution

As with HRNN, start with the solution first.

Note the solution is just a label kind of identifier, you'll also need to create a version which is the actual trained model.

In [8]:
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:924376141954:solution/personalize-poc-sims",
  "ResponseMetadata": {
    "RequestId": "6ef89cea-c35f-4c0b-9cff-37663ea4f696",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 15 Feb 2020 13:30:27 GMT",
      "x-amzn-requestid": "6ef89cea-c35f-4c0b-9cff-37663ea4f696",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the Solution Version

This process will actually take a while to complete, upwards of 25 minutes on. Normally there would be while loops to poll until the task is completed. However the task would block other cells from executing and the goal here is to create many models and deploy them quickly. Below there are instructions to viewing the progress in browser. After creating all of the solution versions go there and watch for updates.

In [10]:
sims_create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [15]:
sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
print(json.dumps(sims_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:924376141954:solution/personalize-poc-sims/c283def0",
  "ResponseMetadata": {
    "RequestId": "7de2d588-7251-484d-a5ab-a3e6e30b8121",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 15 Feb 2020 13:31:22 GMT",
      "x-amzn-requestid": "7de2d588-7251-484d-a5ab-a3e6e30b8121",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Personalized Ranking

Personalized Ranking is an interesting application of HRNN. Instead of just recommending what is most probable for your user in question, this algorithm takes in a user and a list of items as well. The items are then rendered back in the order of most probability for the user. The use case here is for filtering on genre for example, or when you have a broad collection that you would like better ordered for a particular user.

#### Select Recipe

In [11]:
#### Personalized-rerank
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### Create and Wait for Solution
First you will create the solution with the API, then you will create a version. 

Note the solution is just a label kind of identifier, you'll also need to create a version which is the actual trained model.

In [12]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-poc-rerank",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:924376141954:solution/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "5ea3b62d-c193-43f8-baf6-5b940b3a85ed",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 15 Feb 2020 13:31:31 GMT",
      "x-amzn-requestid": "5ea3b62d-c193-43f8-baf6-5b940b3a85ed",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the Solution Version

This process will actually take a while to complete, upwards of 25 minutes on. Normally there would be while loops to poll until the task is completed. However the task would block other cells from executing and the goal here is to create many models and deploy them quickly. Below there are instructions to viewing the progress in browser. After creating all of the solution versions go there and watch for updates.

In [13]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [16]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:924376141954:solution/personalize-poc-rerank/3866a90f",
  "ResponseMetadata": {
    "RequestId": "93edcd94-fb92-494f-a58e-05543d424522",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 15 Feb 2020 13:31:41 GMT",
      "x-amzn-requestid": "93edcd94-fb92-494f-a58e-05543d424522",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Viewing Solution Creation Status

As promised, how to view the status updates in the console:

* In another browser tab you should already have the AWS Console up from opening this notebook instance. 
* Switch to that tab and search at the top for the service `Personalize`, then go to that service page. 
* Click `View dataset groups`.
* Click the name of your dataset group, most likely something with POC in the name.
* Click `Solutions and recipes`.
* You will now see a list of all of the solutions you created above. Click any one of them. 
* Note in `Solution versions` the job that is in progress. Once it is `Active` you solution is ready to be reviewed. It is also capable of being deployed.


## Evaluating Solutions

After about an hour max the solutions should be ready for review. While they are in progress it is a good idea to cover the various algorithms and their behavior in depth. You'll have another lull period as the solutions are being deployed into campaigns as well, so you can split the material into 2 sections if that makes it easier. Also it can be a good time to discuss alternatives to how the data was fed into the system and what kind of results to expect from it.

The firt step is to obtain the solutions metrics, API calls for each below.

### HRNN Metrics:

In [17]:
hrnn_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_solution_version_arn
)

print(json.dumps(hrnn_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:924376141954:solution/personalize-poc-hrnn/00987cb2",
  "metrics": {
    "coverage": 0.0407,
    "mean_reciprocal_rank_at_25": 0.0316,
    "normalized_discounted_cumulative_gain_at_10": 0.0408,
    "normalized_discounted_cumulative_gain_at_25": 0.0545,
    "normalized_discounted_cumulative_gain_at_5": 0.0323,
    "precision_at_10": 0.0066,
    "precision_at_25": 0.0053,
    "precision_at_5": 0.0077
  },
  "ResponseMetadata": {
    "RequestId": "bd13ee02-2080-4d92-9d8e-bd98b1e15b1f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 15 Feb 2020 14:30:44 GMT",
      "x-amzn-requestid": "bd13ee02-2080-4d92-9d8e-bd98b1e15b1f",
      "content-length": "404",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


The above metrics tell us that at 5 items, we have less than a 1% chance (.8% literally) in a recommendation being a part of a user's interaction history (in the hold out phase from training and validation). This is clearly not a great model, but keep in mind instead of plays, all we had to go on is that they tagged a particular artist and that is it.

Something else to note, the users' history is influenced by ANY recommendation model that is in place as your historical data is being collected. This often means that while your model probably won't be this bad with a customer or on your own data, it does bias the metrics to favor their existing solution. If you work to just push the offline metrics to match or exceed their existing solution you may just be making HRNN start to behave like whatever they were already using.

This is a great time to have a conversation about AB testing and to think about the actual business outcomes they are looking to drive. From there you look to run small experiments with Personalize against their existing recommendation system and see over time how the AB test performs. If Personalize is winning then it is the time to filter more and more traffic to Personalize and campaigns within it. Over time the bias from the existing model will fade.

### SIMS Metrics:

In [18]:
sims_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(sims_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:924376141954:solution/personalize-poc-sims/c283def0",
  "metrics": {
    "coverage": 0.0846,
    "mean_reciprocal_rank_at_25": 0.0393,
    "normalized_discounted_cumulative_gain_at_10": 0.0444,
    "normalized_discounted_cumulative_gain_at_25": 0.0571,
    "normalized_discounted_cumulative_gain_at_5": 0.036,
    "precision_at_10": 0.0133,
    "precision_at_25": 0.0099,
    "precision_at_5": 0.015
  },
  "ResponseMetadata": {
    "RequestId": "ab0a81be-feba-4056-8fb1-b4b94ca73e70",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 16 Feb 2020 10:27:56 GMT",
      "x-amzn-requestid": "ab0a81be-feba-4056-8fb1-b4b94ca73e70",
      "content-length": "402",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In this example we are seeing a slightly elivated precision at 5, a little over 1% at 1.04% this time. Effectively this is probably within the margin of error but given that no effort was made to mask popularity, it may just be returning super popular results that a large volume of users have interacted with in some way. 

### Personalized Ranking Metrics:

In [19]:
rerank_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)

print(json.dumps(rerank_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:924376141954:solution/personalize-poc-rerank/3866a90f",
  "metrics": {
    "coverage": 0.0021,
    "mean_reciprocal_rank_at_25": 0.0193,
    "normalized_discounted_cumulative_gain_at_10": 0.0345,
    "normalized_discounted_cumulative_gain_at_25": 0.0461,
    "normalized_discounted_cumulative_gain_at_5": 0.0293,
    "precision_at_10": 0.0063,
    "precision_at_25": 0.0053,
    "precision_at_5": 0.0091
  },
  "ResponseMetadata": {
    "RequestId": "0fa00a0a-5304-490c-8f89-cafaf2cad503",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 16 Feb 2020 10:27:58 GMT",
      "x-amzn-requestid": "0fa00a0a-5304-490c-8f89-cafaf2cad503",
      "content-length": "406",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Storing Useful Variables]

Before exiting this notebook, run the following cells to save off our version arns for use later.

In [20]:
%store hrnn_solution_version_arn
%store sims_solution_version_arn
%store rerank_solution_version_arn

Stored 'hrnn_solution_version_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'rerank_solution_version_arn' (str)



Congratulations. You have created 3 solutions on Amazon Personalize and analyzed their performance.